<a href="https://www.kaggle.com/code/gpreda/gemini-1-5-q-a-from-a-large-romanian-book?scriptVersionId=209256926" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>


<center><img src="https://i0.wp.com/historice.ro/wp-content/uploads/2014/02/Lancelot-casa-din-Bucuresti.jpg" width=800</img></center>

# Introduction


In this Notebook we will demonstrate how to use **Gemini 1.5 flash** to query a large book from the Romanian literature.  The book, "Scrisori catre Vasile Alecsandri" (Letters to Vasile Alecsandri) by Ion Ghica, is a classical of Romanian literature.



## Step 1: Import Python Packages

In [1]:
import os
import time
import google.generativeai as genai
from kaggle_secrets import UserSecretsClient
from time import time

## Step 2: Authenticate with Google Generative AI

In [2]:
user_secrets = UserSecretsClient()
ai_studio_token = user_secrets.get_secret("GEMINI_API_KEY_SECOND")
genai.configure(api_key=ai_studio_token)

## Step 3: Define helper functions

In [3]:
def upload_to_gemini(path, mime_type=None):
  """Uploads the given file to Gemini.

  See https://ai.google.dev/gemini-api/docs/prompting_with_media
  """
  file = genai.upload_file(path, mime_type=mime_type)
  print(f"Uploaded file '{file.display_name}' as: {file.uri}")
  return file

def wait_for_files_active(files):
  """Waits for the given files to be active.

  Some files uploaded to the Gemini API need to be processed before they can be
  used as prompt inputs. The status can be seen by querying the file's "state"
  field.

  This implementation uses a simple blocking polling loop. Production code
  should probably employ a more sophisticated approach.
  """
  print("Waiting for file processing...")
  for name in (file.name for file in files):
    file = genai.get_file(name)
    while file.state.name == "PROCESSING":
      print(".", end="", flush=True)
      time.sleep(10)
      file = genai.get_file(name)
    if file.state.name != "ACTIVE":
      raise Exception(f"File {file.name} failed to process")
  print("...all files ready")
  print()

## Step 4: Load the Gemini 1.5 model

In [4]:
# Create the model
generation_config = {
  "temperature": 0.5,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

## Step 5: Upload your large file to Gemini 1.5

In [5]:
romanian_book = "/kaggle/input/scrisori-ctre-vasile-alecsandri-de-ion-ghica/Scrisori_catre_Vasile_Alecsandri.pdf"

In [6]:
files = [
  upload_to_gemini(romanian_book, mime_type="application/pdf"),
]

wait_for_files_active(files)

chat_session = model.start_chat(
  history=[
    {
      "role": "user",
      "parts": [
        files[0],
      ],
    }
  ]
)

Uploaded file 'Scrisori_catre_Vasile_Alecsandri.pdf' as: https://generativelanguage.googleapis.com/v1beta/files/kh5nudtksmwj
Waiting for file processing...
...all files ready



## Step 6: Ask Gemini 1.5 questions about your large file

In [7]:
def query_model(prompt):
    start_time = time()
    response = chat_session.send_message(prompt)
    print(response.text)
    print(response.usage_metadata)
    end_time = time()
    print(f"Total query time: {round(end_time - start_time, 2)} sec.")

<center><img src="https://i0.wp.com/historice.ro/wp-content/uploads/2014/02/Lancelot-romani-din-Timisoara.jpg" width=800</img></center>

In [8]:
query_model("Make a summary of this book, and write this summary in French. Do it in less than 1000 typograpical signs")

Voici un résumé du livre "Scrisori către Vasile Alecsandri" d'Ion Ghica, en français et en moins de 1000 signes :

Ce recueil de lettres, adressées à Vasile Alecsandri, couvre une période de la vie de l'auteur, Ion Ghica, marquée par des changements politiques et sociaux importants en Roumanie.  Ghica évoque des anecdotes, des portraits, des réflexions sur la société roumaine, la liberté, l'égalité, et le rôle de la presse et de la littérature.  Le ton est personnel et anecdotique, offrant un aperçu intime et vivant de l'époque.  Le livre explore la transition de la Roumanie, ses défis et ses espoirs, à travers le regard d'un témoin privilégié.

prompt_token_count: 421698
candidates_token_count: 166
total_token_count: 421864

Total query time: 39.12 sec.



<center><img src="https://i0.wp.com/historice.ro/wp-content/uploads/2014/02/Lancelot-sat-langa-Targu-Jiu.jpg" width=800</img></center>

In [9]:
query_model("Make the summary of the first chapter in the book. Do it in maximum 3 phrases. Write this answer in German.")

Das erste Kapitel des Buches bietet eine Einführung in die Briefe von Ion Ghica an Vasile Alecsandri.  Es beschreibt den historischen Kontext der Korrespondenz, die im 19. Jahrhundert während einer Zeit tiefgreifender Veränderungen in der rumänischen Gesellschaft stattfand.  Der Leser erhält einen Überblick über die Themen, die in den folgenden Briefen behandelt werden.

prompt_token_count: 421891
candidates_token_count: 77
total_token_count: 421968

Total query time: 38.0 sec.



<center><img src="https://i0.wp.com/historice.ro/wp-content/uploads/2014/02/Lancelot-bordei-din-Campia-Dunarii.jpg" width=800</img></center>

In [10]:
query_model("Summarize the chapter where is described a journey from Bucharest, the capital of Valachia to Iassi, the capital of Moldavia. Do it in maximum 10 phrases. Do it in Japanese.")

ブカレストからヤシへの旅を描いた章では、19世紀半ばのワラキア公国とモルドバ公国の様子が生き生きと描写されています。旅の道中は、要塞都市や村々を通過し、当時の社会情勢や人々の生活を垣間見ることができます。道は悪く、危険も潜んでいました。旅は困難を極め、多くの時間を要しました。ダニューブ川沿いの風景や、ドナウ川を渡る様子なども描かれています。著者は、様々な人物との出会いや出来事を詳細に記しています。当時の社会構造や政治状況についても触れられています。この旅の描写を通して、当時のルーマニアの現実が浮き彫りになります。

prompt_token_count: 422012
candidates_token_count: 155
total_token_count: 422167

Total query time: 119.66 sec.



<center><img src="https://i0.wp.com/historice.ro/wp-content/uploads/2014/02/Fantana-langa-manastirea-Argesului.jpg" width=600</img></center>

# Conclusions

The model answers correctly to a variety of questions about a book in Romanian language in a number of different languages (French, German, and Japanese).   
The answer time is around a minute for each of the answers.

Credit:
 - Adapted from https://aistudio.google.com/app/prompts/video-qa